In [ ]:
# CS542 Spring 2024, sections 4 & 5
# Notebook 1
# Nik Sultana, Illinois Institute of Technology
#
# For the FABlib API, see: https://fabric-fablib.readthedocs.io/en/latest/index.html


# NOTE Review how to upload & download from JupyterHub. You'll start by uploading this notebook.


from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

new_slice = False
if 0 == len(fablib.get_slices()):
    new_slice = True
    image = 'default_ubuntu_20'
    site = fablib.get_random_sites(count = 1)[0]
    slice = fablib.new_slice(name = "cs542_slice")
    n1 = slice.add_node(name = "n1", image = image, cores = 2, ram = 4, disk = 50, site = site)
    n1_iface1 = n1.add_component(model = "NIC_Basic", name = "iface1").get_interfaces()[0]

    n2 = slice.add_node(name = "n2", image = image, cores = 2, ram = 4, disk = 50, site = site)
    n2_iface1 = n2.add_component(model = "NIC_Basic", name = "iface1").get_interfaces()[0]

    net = slice.add_l2network(name = "net", interfaces=[n1_iface1, n2_iface1], type = "L2Bridge")
    slice.submit()

    #  |--------|                        |--------|
    #  |        |                        |        |
    #  |   n1   |iface1)--(net)---(iface1|   n2   |
    #  |        |                        |        |
    #  |________|                        |________|

elif 1 == len(fablib.get_slices()):
    slice = fablib.get_slices()[0]
    slice.show()
    for node in slice.get_nodes():
      node.show()
    for net in slice.get_networks():
      net.show()
    for iface in slice.get_interfaces():
      iface.show()

else: assert(len(fablib.get_slices()) <= 1)


# NOTE Look at the slice through the FABRIC Portal -- through its non-API interface.
#      You can also delete a slice from there.

slice.get_lease_end() # By default, your slice will disappear after 24 hours.

In [ ]:
# We can also execute commands on n1 from the notebook
n1 = slice.get_node(name = "n1")
n2 = slice.get_node(name = "n2")

if new_slice:
    threads = []
    threads.append(n1.execute_thread("sudo apt update -y -qq && sudo apt install -y build-essential && sudo apt install -y net-tools && sudo apt install -y python3-scapy"))
    threads.append(n2.execute_thread("sudo apt update -y -qq && sudo apt install -y build-essential && sudo apt install -y net-tools && sudo apt install -y python3-scapy"))
    for thread in threads:
        print(thread.result())

In [ ]:
print("ssh works on n1: " + str(n1.test_ssh()))
print(n1.get_ssh_command())
print("ssh works on n2: " + str(n2.test_ssh()))
print(n2.get_ssh_command())

# Try ssh'ing into n1 and n2
# And run `sudo apt install -y tshark` -- this insists on an interactive question, so we cannot batch it with the others.

# Now that the software is set up, let's configure the experiment network.

In [ ]:
# Management vs Experiment interfaces.
print("ifconfig on n1:")
n1.execute("ifconfig")
print("ifconfig on n2:")
n2.execute("ifconfig")

print("Are those the 2 interfaces we created?")

# Over ssh, try `ifconfig -a`

In [ ]:
from ipaddress import IPv6Address, IPv6Network

n1_iface1 = n1.get_interface(name = "n1-iface1-p1")
n2_iface1 = n2.get_interface(name = "n2-iface1-p1")
# Use local IPv6 address -- see RFC 4193
n1_iface1.ip_addr_add(addr = IPv6Address("fd3f:f209:c711::1"), subnet = IPv6Network("fd3f:f209:c711::/48"))
n2_iface1.ip_addr_add(addr = IPv6Address("fd3f:f209:c711::2"), subnet = IPv6Network("fd3f:f209:c711::/48"))
n1_iface1.show()
n2_iface1.show()

print("Experiment interfaces now have addresses")

In [ ]:
n1.execute("sudo ifconfig " + n1_iface1.get_device_name() + " up")
n2.execute("sudo ifconfig " + n2_iface1.get_device_name() + " up")
# Note can also use n1_ifaceX.ip_link_up()

print("ifconfig on n1:")
n1.execute("ifconfig")
print("ifconfig on n2:")
n2.execute("ifconfig")

In [ ]:
command = "ping -c 1 -I " + n1_iface1.get_device_name() + " fd3f:f209:c711::2"
print(command) # We can copy-and-paste this into a shell on n1
n1.execute(command) # Or execute it from here

#Or equivalently: n1.execute("ping -c 1 -I enp7s0 fd3f:f209:c711::2")

In [ ]:
# Transferring files between your JupyterHub and your node

# NOTE could use n1.upload_file(...)
def cmd_upload_file_to(n, filename):
    scp_pre_command = (n.get_ssh_command()).split(" ")
    scp_pre_command[0] = 'scp'
    scp_pre_command[5] = scp_pre_command[5].split("@")
    scp_pre_command[5] = scp_pre_command[5][0] + "@[" + scp_pre_command[5][1] + "]:~/"
    scp_pre_command.insert(5, filename)
    return ' '.join(scp_pre_command)

def cmd_download_file_from(n, filename):
    scp_pre_command = (n.get_ssh_command()).split(" ")
    scp_pre_command[0] = 'scp'
    scp_pre_command[5] = scp_pre_command[5].split("@")
    scp_pre_command[5] = scp_pre_command[5][0] + "@[" + scp_pre_command[5][1] + "]:~/" + filename
    scp_pre_command.append(".")
    return ' '.join(scp_pre_command)

print(cmd_upload_file_to(n1, "Readme.md"))
print(cmd_download_file_from(n1, "somefile"))

In [ ]:
# Next:
# 1) Using tcpdump and tshark
#    Listening to an interface "live" and analyzing traffic.
#    Saving a packet capture (pcap) file from a listening session.
#    Analyzing a pcap file.
# 2) "Replaying" traffic
# 3) Using Scapy
#    Listening to an interface "live" and analyzing traffic.
#    Saving a packet capture (pcap) file from a listening session.
#    Analyzing a pcap file.
#    Editing a pcap file.
#    Creating new packets into a pcap file or "live" traffic.

In [ ]:
slice.delete()